In [ ]:
#requirements.txt
#https://github.com/alexklibisz/elastiknn/blob/main/examples/tutorial-notebooks/multimodal-search-amazon-products.ipynb
#https://towardsdatascience.com/computing-node-embedding-with-a-graph-database-neo4j-its-graph-data-science-library-d45db83e54b6

#python -m venv .venv
#source .venv/bin/activate
#pip install -r requirements.txt
#jupyter lab
###work work work
###deactivate

#docker-compose up

In [ ]:
####Alternative setup for leveraging RAPIDS-AI conda env
#ubuntu-drivers devices
#sudo add-apt-repository ppa:graphics-drivers/ppa --yes
#sudo apt update
#watch -n 1 nvidia-smi

# To activate this environment, use                                             
#                                                                               
#     $ conda activate rapids-22.02                                             

#pip install -r requirements.txt
#                                                                               
# To deactivate an active environment, use                                      
#                                                                               
#     $ conda deactivate


In [ ]:
#%pip install cudf
#%conda install -c rapidsai -c conda-forge -c nvidia \
#    cudf python=3.10 cudatoolkit=11.8

In [ ]:
#RSN 20 - Please Pin cuda-python to 11.7.0
#Resolution Path 1: You can upgrade your RAPIDS version to `v22.06.01`.  
#- We've made it easy to upgrade: simply use our Release selector and install `v22.06` or rerun you `v22.06` installation script. It will automatically grab `v22.06.01` via Conda or Docker.

#Resolution Path 2: If you can NOT upgrade, pin `cuda-python` to `11.7.0` in RAPIDS `v22.02`, `v22.04`, and `v22.06`, 
#- You can pin `cuda-python` to `11.7.0` by using `cuda-python=11.7.0` when doing an install.  
#- Example command: `conda install -c nvidia cuda-python=11.7.0`.

In [ ]:
#progressbar
#%pip install tqdm                    # pip
#%pip install ipywidgets
#jupyter nbextension enable --py widgetsnbextension


In [1]:
import cudf

In [3]:
%pip install faiss-gpu --quiet

Note: you may need to restart the kernel to use updated packages.


In [74]:
#%pip install dask-cuda --quiet

ERROR: Cannot uninstall 'locket'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%pip install opensearch-py==2.2.0

In [ ]:
#%pip install graphviz --quiet

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from vectordocutil import *
from itertools import islice
#from tqdm import tqdm
from IPython.display import Image, display, Markdown, Code, HTML
import matplotlib.pyplot as plt
import numpy as np
import json
from datetime import datetime
import random

#for PoC on Milvus vector DB
#from pymilvus import CollectionSchema, FieldSchema, DataType

import faiss
import torch
from sentence_transformers import SentenceTransformer
from faker import Faker

from dask import delayed
from time import sleep

from tqdm.notebook import tqdm, trange

In [2]:
#To validate presence of GPU card
#watch -n 1 nvidia-smi
import cudf
!nvidia-smi

Tue Apr 25 18:21:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P0    N/A /  N/A |    539MiB /  4040MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## To make OpenSearch up-running

In [ ]:
#docker compose for OpenSearch
#https://opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/
#Setup for OpenSearch
#https://opensearch.org/downloads.html
#1. Set up your Docker host environment
#    * macOS & Windows: In Docker Preferences > Resources, set RAM to at least 4 GB.
#    * Linux: Ensure vm.max_map_count is set to at least 262144 as per the documentation.
#2. Download docker-compose.yml into your desired directory
#3. Run docker-compose up
#4. Have a nice coffee while everything is downloading and starting up
#5. Navigate to http://localhost:5601/ for OpenSearch Dashboards
#6. Login with the default username (admin) and password (admin)

%/docker/docker-compose up -d
%/docker/docker-compose ps 
%/docker/docker-compose down -v
#OpenSearch Dashboard
#http://localhost:5601/app/dev_tools#/console

#Elasticsearch: OpenSarch, OpenDistro, Elastic
#https://www.elastic.co/guide/en/elasticsearch/client/python-api/master/connecting.html
#https://elasticsearch-dsl.readthedocs.io/en/latest/search_dsl.html
#https://elasticsearch-dsl.readthedocs.io/en/latest/index.html
#curl -XGET https://localhost:9200 -u admin:admin --insecure  
#https://github.com/elastic/elasticsearch-py/issues/712
#curl -XGET https://localhost:9200/_cat/indices -u admin:admin --insecure 

#this does work on mac terminal, not via Jupyter Notebook
#https://elasticsearch-py.readthedocs.io/en/v8.5.2/
#https://github.com/pgolding/elasticsearch/blob/master/index.py
#to kill service ginve a port
##sudo kill -9 $(sudo lsof -t -i:5601)

## To define GPU for accelerating the SentenceTransformer encoding tasks

In [10]:
#torch_device = 'cpu'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

#SentenceTransformer models to validate
#model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=torch_device)
#model = SentenceTransformer('paraphrase-MiniLM-L6-V2', device=torch_device)

model = SentenceTransformer('all-MiniLM-L6-v2',
                              device=torch_device)

print(torch_device)

#loading and validating Faker text generator
fake = Faker(['en_US'])
print(fake.text())

cuda
Catch under red tax anything provide tough strategy. Recognize view art may event test data. Seven by final enjoy really.


## Connect to Neo4j for extracting embeddings out of the graph via GDS

In [ ]:
#Loading from Neo4j trees/graph representation of OCID's branches as vector embeddings
#fastRP
#to validate GDS Neo4j library, fastRP
#https://neo4j.com/developer/graph-data-science/applied-graph-embeddings/
#https://pytorch-geometric.readthedocs.io/en/latest/advanced/remote.html

#... To import the dataset, run the followin -> Nodes: Place, Country; Edges: EROAD, IN_COUNTRY
#CALL gds.graph.project(
#  'places_undir',
#  'Place',
#  {EROAD: {orientation: 'UNDIRECTED'}}
#)

#CALL gds.fastRP.stream('places_undir',
#  {
#    embeddingDimension: 10
#  }
#)
#YIELD nodeId, embedding
#RETURN gds.util.asNode(nodeId).name AS place, embedding
#LIMIT 5


## Connect to Elasticsearch/OpenSearch

In [20]:
#https://dylancastillo.co/opensearch-python/
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

#validate connection with basic query
idx = "h_ocid"
resp = client.search(
    index = idx,
    body={
          "query": {
            "match": {
              "doc": "morning" 
            }
          },            
    }
)
resp

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

## Dask pipelining leveraging @delayed

In [ ]:
#https://examples.dask.org/delayed.html
#https://examples.dask.org/applications/evolving-workflows.html
#    pipelining with dask?
#    https://examples.dask.org/applications/prefect-etl.html
#       https://docs.prefect.io/getting-started/installation/
#    pipeline https://examples.dask.org/delayed.html


#Delaied pipelining concept
from dask import delayed

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

@delayed
def incD(x):
    sleep(1)
    return x + 1

@delayed
def addD(x, y):
    sleep(1)
    return x + y

%%time

x = inc(1)
y = inc(2)
z = add(x, y)

x = delayed(incD)(1)
y = delayed(incD)(2)
z = delayed(addD)(x, y)


z.compute()

z.visualize()

## To tackle large size query arguments passing from one app domain to another (A->B)
### Side Effects/limitations in such scenario (two steps query):
1. As larger the the size of the data/payload being transferred A->B, worse the overall performance (IO, processing time)
2. Protocols/Drivers/Query engines eventually limit the size of input arguments
    2.1 Elastic limits in 65k input args
    2.2 Index/keyword database does not implement "join" on indexes
3. Have to define a cut (by which rule?). Can not manage *unlimited size of arguments passing A->B

### Assumption: (Exact match vs approximation)
Must to understand users behavior: If strictly only exact match queries matters. Otherwise, query by approximation can for sure be a solution. 
*New CAP Theorem:
CAP(Consistency, Availability, Partition Tolerance) vs CAP(Cost, Accuracy, Performance)

### QKnows scenario:
1. Search pipeline
2. Subtree expansion



### Solution (one step query):
Leverage vector embeddings similarity instead of exact-match
Less complexity: self-contained data for keyword and vector

Pipelines:
1. On query time: (take as input argument the leaf+one before OCID) ~> (convert hierarchy to string) ~> (Encode/Vectorize) ~> (pass to knn_vector attribute)
2. Preprocessing: (take upper parent nodes to root) ~> (convert hierarchy to string) ~> (Encode/Vectorize) ~> (persist intto knn_vector attribute doc)

*Scores higher by taking at least two ocids (the leaf+one before)

### Concern
1. Evaluate datasets to find best size of the vector space which best capture meaning assuming the lenght of a OCID path in the Ontology tree is virtually unlimited: As higher the lenght as harder to a small vector space to capture the meaning.

### Alternative solutions:
1. (Two steps query) to avoid input encoding on query time: Preprocess ocid paths to all nodes (ocid) to extract their own hash->embedding and store in a Key-value database, annotate docs with that new branded representation for subtree expansio. Results in a two step

### Efficient tech solution
To avoid two steps application query and also power complex queries, best technology would be based on self contained doc-Index/keyword and Vector Similarity Databases: via plugin (OpenSearch, Elasticsearch),...

Other vector databases: Milvus, Vespa, *weaviate

### Yet another CAP Theorem to consider:
Milvus: A Vector Database For Unstructured Data Processing
https://youtu.be/_x8RTSaiT50?t=441

### Next Steps
1. Performe encode embedding on the concept text instead of OCID number
2. Train a SentenceTransformer model with our own dataset
3. Performe encode embedding on OCID/Concet with a Graph Embeddings algorithm 

## Functions data pipeline to load doc/embeddings to OpenSearch {listToJoinedWords ~> stringToVector ~> vectorEnhancedDoc ~> sendToSearch}

In [8]:
#https://www.bmc.com/blogs/elasticsearch-commands/
#https://opensearch.org/docs/1.0/search-plugins/knn/approximate-knn/
#https://medium.com/@bb8s/embedding-based-retrieval-approximate-nearest-neighbor-algorithms-used-in-production-systems-b96dd4b2e9a3
#https://github.com/UKPLab/sentence-transformers/issues/1319
#https://github.com/ueg1990/faker-schema
import json

#Pipeline
#    Load json file in a collection
#    Function returns yield item ocid and string joined ocid’s
#    Function returns yield item ocid and vectorized string
#    Function returns yield item ocid and Fake doc{ +vectorized string}
#    Iteration over last item to send each item to OpenSearch and append final doc to a collection
#    Operation to save as json the collection
#    

#load json OCID's into collection
f = open('data/data_tree.json')
data = json.load(f) #OCID->[OCID]
f.close()

#connecting to OpenSearch
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False, 
    timeout=60
)
client.info()

###################Task-functions to the pipeline###################

#function to convert key-list into key-(string:joined ocid words)
def listToJoinedWords(ls):
    r = " ".join([str(x) for x in ls])
    #print(ls)
    return r

#function to convert string: joined ocid words into vectorized embedding representation
def stringToVector(md, sentence, t_device):
    ret = md.encode(sentence,device=t_device)
    #ret = np.random.random(768)
    #print(sentence)
    #print(md)
    #print(t_device)
    return ret
    
#function receives embedded representation and creates the document for being later sent to elastic
def vectorEnhancedDoc(key, vector, faker, index, ocids):
    doc = {
            "my_vector": vector,
            "doc": faker.text(),
            "ocids": ocids
    }
    return doc

def vectorEnhancedDoc_2(key, vector1, vector2, faker, index, ocids1, ocids2):
    doc = {
            "ocid_vec1": vector1,
            "ocid_vec2": vector2,        
            "doc": faker.text(),
            "ocids_1": ocids1,
            "ocids_2": ocids2        
    }
    return doc

def sendToSearch(index, key, doc):
    #resp = es.index(index=idx, id=key, document=doc)
    #print(doc)
    client.index(index=index, id=key, body=doc)
    1
###################Task-functions to the pipeline###################

#todo
#routine to create a collection for later save to disk key-doc data structure. Facilitate the test
#OCID->DOC{[OCID].toString().toVector} json.save

## Running the pipeline

In [147]:
#https://docs.dask.org/en/stable/best-practices.html
#https://distributed.dask.org/en/stable/memory.html

#Creating the data chunks pagination mechanism
ifrom = 0
ito = len(data)

d2 = list(data.items())[ifrom:ito]

pagesize = 42

pagedData = [d2[i:i+pagesize] for i in range(0, len(d2), pagesize)]

#pagedData
#mylist = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
#a = [mylist[i:i+2] for i in range(0, len(mylist), 2)]
#dict(pagedData[1])

In [149]:
#%%timeit
import time
start = time.time()

import dask

#Dask delayed functions
listToJoinedWordsD = dask.delayed(listToJoinedWords)
stringToVectorD = dask.delayed(stringToVector)
vectorEnhancedDocD = dask.delayed(vectorEnhancedDoc)
sendToSearchD = dask.delayed(sendToSearch)

print("Batches: " + str(len(pagedData)))

#Pages/data chunks mapping to how many DAGs DASK can run in parallel/assync 
index = "h_ocid_3"
for i, chunck in zip(trange(len(pagedData)), pagedData):
    dchunk = dict(chunck)
    zs = []
    for key in dchunk: 
        x = listToJoinedWordsD(data[key])
        y = stringToVectorD(model, x, 'cpu')
        z = vectorEnhancedDocD(key,y,fake,index, data[key])
        send = sendToSearchD(index, key, z)
        zs.append(send)
        #send.visualize(filename="transpose_opt.svg", optimize_graph=True)
    zs = dask.compute(*zs)  # trigger computation in the background


end = time.time()
print('Total time:', (end - start)/60)

#encode with gpu: 44.2 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#encode with cpu: 1min 24s ± 1.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Batches: 1477


  0%|          | 0/1477 [00:00<?, ?it/s]

Total time: 78.94896431763966


In [146]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## TODO: Data Validation: Precision, Recall & Accuracy

## Basic query validations/evaluations REPL

In [ ]:
#https://opensearch.org/docs/1.0/search-plugins/knn/approximate-knn/
#to create the index
DELETE /h_ocid

PUT /h_ocid
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 256,
              "m": 48
            }
        }
      },
    "doc": { "type": "text" },
    "ocids": { "type": "text" }
    }
  }
}

GET h_ocid/_search
{
  "query": {
    "match": {
      "doc": "morning" 
    }
  }
}

## To create adhoc doc (text + vector) to send or query on OpenSearch

In [150]:
#To create adhoc doc (text + vector) to send to OpenSearch

#sentences = ['This framework generates embeddings for each input sentence',
#    'Sentences are passed as a list of string.', 
#    'The quick brown fox jumps over the lazy dog.']

#df_nested_list[df_nested_list['key'].isin(['241000592342','241000592344','241000607358','241000531115','241000601264'])]

index = "h_ocid"

key = 241000592342

#ocids = [ 241001562097, 241001562098, 241001562102]
ocids = [ 241001562102]

sentences = listToJoinedWords(ocids)

#sentence_embeddings = model.encode(sentences, device=torch_device)
vector = stringToVector(model, sentences, 'cpu')

doc = vectorEnhancedDoc(key,vector, fake, index, ocids)


#Sending to OpenSearch
#sendToSearch(index, key, doc)

doc

#for sentence, embedding in zip(sentences, sentence_embeddings):
#    print("Sentence:", sentence)
#    print("Embedding:", embedding)
#    print("")
    

#testing json cuDF loader - gpu-accelerated-json-data-processing-with-rapids
#https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.read_json.html
#https://github.com/rapidsai-community/showcase/blob/main/getting_started_tutorials/10min_to_cudf_colab.ipynb?nvid=nv-int-tblg-386840-vt27#cid=av02_nv-int-tblg_en-us
#https://developer.nvidia.com/blog/gpu-accelerated-json-data-processing-with-rapids/

{'my_vector': array([-8.91607627e-02, -3.25781778e-02, -1.40374586e-01, -7.52803609e-02,
        -9.17210728e-02,  3.91724184e-02,  1.88997447e-01,  1.19780645e-01,
        -1.03507839e-01, -1.35149628e-01, -2.86350846e-02,  2.66810089e-01,
        -7.70434141e-02, -2.23531991e-01,  2.76873782e-02,  2.21521333e-02,
         1.15487427e-01,  5.08065298e-02, -1.05960801e-01, -5.30899223e-03,
         1.89303190e-01, -3.80720035e-03,  2.13753104e-01,  4.92141843e-01,
         2.04584505e-02,  4.65227495e-04,  2.48648465e-01,  1.22591719e-01,
        -2.44414434e-01, -9.74549577e-02,  8.90667439e-02, -1.99114934e-01,
         2.45968550e-01, -3.05224687e-01, -2.16461085e-02, -6.09709695e-02,
        -2.81338781e-01, -1.35938570e-01,  1.50033072e-01,  8.12813416e-02,
         5.07899262e-02, -2.99041957e-01,  1.16858326e-01, -7.40455016e-02,
        -1.21233143e-01,  1.95570067e-02, -3.71416032e-01,  1.95153058e-01,
        -2.88928356e-02, -2.53924765e-02,  4.20807954e-03, -1.32645860e-01,

## Query on OCID dataset

In [57]:
import pandas as pd
from collections import OrderedDict
import json

json_ocid = 'data/data_tree.json'
parquet_ocid = 'data/df_ocid_path-size.parquet.gzip'

with open('data/data_tree.json','r') as f:
    jdata = json.loads(f.read())

#jdata

In [58]:
# Flatten data
df_nested_list = pd.json_normalize(jdata).transpose()
df_nested_list.reset_index(inplace=True)
df_nested_list.columns = ['key','ocids']

df_nested_list

,key,ocids
0,190035525634,[190070311615]
1,190001971203,[190074031319]
2,190014554116,[190014554115]
3,190039719941,"[190005544317, 190005675461]"
4,241000513543,"[241000641890, 241001116483, 241000012834, 241..."
...,...,...
62009,190039457615,[190005566535]
62010,190009289361,"[190006521360, 190006521362]"
62011,190039457663,[190000739902]
62012,241000120264,"[241000120249, 241000121426, 241000121398]"


In [62]:
def returnLastNList(ls, n):
    #list slicing S2
    return list(ls)[-n:]

In [59]:
#adding a new column as the len of the ocids: How many OCIDs in a root/branch ontology?
df_nested_list['ocid_len'] = df_nested_list.apply(lambda row: len(row.ocids), axis = 1)

df_nested_list['last3OCIDs'] = df_nested_list.apply(lambda row: returnLastNList(row.ocids,3), axis = 1)

df_nested_list.to_parquet(parquet_ocid, compression='gzip')

In [60]:
df_nested_list.sort_values(by='ocid_len', ascending=False)

,key,ocids,ocid_len,last3OCIDs
39955,241000592342,"[241001562115, 241001562130, 241001562133, 241...",184071,"[241001562097, 241001562098, 241001562102]"
39956,241000592344,"[241001562115, 241001562130, 241001562133, 241...",176867,"[241001562097, 241001562098, 241001562102]"
13793,150000003573,"[190002233346, 190071439364, 190078517254, 190...",148098,"[190072487928, 190005379065, 190014029821]"
16932,150000003819,"[190005379074, 190075240454, 190072487944, 190...",72310,"[190065934326, 190071701501, 190071832574]"
13836,150000003653,"[190050336774, 190017306631, 190050336794, 190...",35253,"[190080548849, 190050336754, 190003412982]"
...,...,...,...,...
25384,190002676657,[190074944290],1,[190074944290]
25386,190011065267,[190010640451],1,[190010640451]
25387,190011065268,[190010882526],1,[190010882526]
25389,190077518777,[190077518763],1,[190077518763]


In [10]:
df_nested_list.describe()

,ocid_len
count,62014.000000
mean,21.841487
std,1271.009819
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,184071.000000


In [48]:
df_nested_list[df_nested_list['key'].isin(['241000592342','241000592344','241000607358','241000531115','241000601264'])]

,key,ocids,ocid_len
8929,241000531115,"[241001546240, 241001546242, 241000895237, 241...",102
39955,241000592342,"[241001562115, 241001562130, 241001562133, 241...",184071
39956,241000592344,"[241001562115, 241001562130, 241001562133, 241...",176867
44711,241000601264,"[241001111043, 241001111044, 241001111045, 241...",227
47212,241000607358,"[241000737156, 241000456452, 241001054857, 241...",55


## Query on OCID dataset leveraging cuDF

In [49]:
import cudf


In [50]:
cuddf_ocid = cudf.read_parquet(parquet_ocid)

In [16]:
cuddf_ocid.describe()

,ocid_len
count,62014.000000
mean,21.841487
std,1271.009819
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,184071.000000


In [17]:
cuddf_ocid.info(verbose=True)

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 62014 entries, 0 to 62013
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   key       62014 non-null  object
 1   ocids     62014 non-null  list
 2   ocid_len  62014 non-null  int64
dtypes: int64(1), list(1), object(1)
memory usage: 12.0+ MB


In [ ]:
cuddf_ocid.sort_values(by='ocid_len', ascending=False)

cuddf_ocid[cuddf_ocid['key'].isin(['241000592342','241000592344','241000607358','241000531115','241000601264'])]
#ValueError: Cannot convert value of type NotImplementedType to cudf scalar

## OpenSearch index definitions/query

In [ ]:
#opensearch query for large list of ocids

_id: 241000592342
POST /h_ocid/_search
{
    "size": 5,
    "_source": [ "doc", "my_vector" ],
    "query": {
        "knn": {
            "my_vector": {
                "vector": [ 5.71955666e-02, -7.66707510e-02, -4.16375212e-02, -6.62729666e-02,
       -4.65770401e-02, -8.40778649e-02, -3.33467796e-02,  5.29849045e-02,
        4.87397313e-02,  5.90010658e-02,  3.52557283e-03,  2.92218681e-02,
        3.02580558e-02,  4.98863459e-02, -4.18894552e-02, -6.16155304e-02,
       -8.42393637e-02, -1.32871285e-01, -3.76987979e-02, -2.37784889e-02,
        2.75300853e-02,  1.57641415e-02,  2.05735932e-03, -4.34888788e-02,
        4.81918938e-02, -3.62182111e-02,  1.38325552e-02,  4.78169210e-02,
       -1.32684261e-02, -5.34942076e-02,  3.49294581e-02,  7.68055245e-02,
        7.58032948e-02, -2.80818865e-02,  7.86984339e-04,  4.89161052e-02,
        1.92692727e-02, -2.52042860e-02,  7.75098726e-02,  4.47247624e-02,
        8.49621892e-02, -6.64084777e-03,  6.99148327e-02,  3.59088220e-02,
        3.40272710e-02, -7.35712871e-02, -1.07011713e-01, -5.85080646e-02,
       -2.95171812e-02, -8.95021409e-02, -1.79006867e-02,  7.73426145e-02,
        7.30564818e-02,  8.58621374e-02, -6.92492723e-02, -1.23154528e-01,
        9.09865927e-03, -3.66995074e-02,  3.36812288e-02,  2.28156038e-02,
       -1.79523267e-02,  6.36661947e-02,  2.02122442e-02, -5.47389090e-02,
        2.94020064e-02,  3.06600868e-03,  3.11299716e-03, -8.18618909e-02,
       -7.44147319e-03,  7.13968724e-02,  4.73366342e-02,  8.63723643e-03,
        2.16671415e-02,  7.52161667e-02,  2.34532673e-02,  1.62310451e-02,
       -4.39223833e-02, -3.74015756e-02,  1.41099410e-03,  3.01953666e-02,
       -1.15964953e-02, -2.99531519e-02,  2.94729043e-02, -6.14674762e-02,
       -1.49631379e-02,  2.71190703e-02,  2.77556460e-02,  7.15614334e-02,
        1.48002151e-02,  4.57992703e-02, -3.69489156e-02,  6.46871689e-04,
       -2.43333224e-02,  1.49020236e-02,  2.01181713e-02,  7.17862789e-03,
        8.71805996e-02,  1.31677128e-02, -6.31324425e-02,  2.75167897e-02,
       -3.35551351e-02, -3.91269103e-02, -1.07200770e-02,  2.38413159e-02,
        5.45703731e-02, -2.97026671e-02,  3.73445675e-02, -1.69750070e-03,
        4.14040609e-04, -9.16920751e-02, -5.67173064e-02, -2.04370380e-03,
        4.54446301e-02, -1.54730817e-03,  1.10675078e-02, -8.41105953e-02,
       -4.71797399e-02,  4.47605476e-02,  4.32032160e-03,  2.93205045e-02,
        1.19888410e-01,  6.15398660e-02, -4.63539138e-02,  5.60447760e-02,
       -3.87760736e-02,  5.20817516e-03, -9.09373462e-02,  1.39064302e-32,
        1.46141124e-03,  5.26829623e-02,  1.16906688e-01,  2.63897981e-03,
       -2.20805090e-02,  1.59152448e-02, -6.30408302e-02, -1.00742383e-02,
       -5.71290962e-02,  6.07439429e-02, -7.41478130e-02, -5.13870046e-02,
        5.26098646e-02,  7.13330647e-03,  7.64036085e-03,  1.62361637e-02,
        3.58895957e-02, -1.72582511e-02, -8.32406878e-02, -9.29833129e-02,
        7.21398517e-02, -2.90668290e-02,  1.75648127e-02, -2.44352557e-02,
        4.40175906e-02,  7.77624026e-02, -6.36347663e-03, -5.42278588e-02,
        5.60236424e-02,  3.24374735e-02, -2.50781253e-02,  4.83281277e-02,
       -1.56093344e-01,  7.14205112e-03,  1.53919458e-02, -7.95859098e-02,
        6.88921660e-02, -9.50438716e-03,  7.58238556e-03, -1.30012520e-02,
        1.93275828e-02,  3.70782502e-02, -4.17744070e-02, -5.24241999e-02,
        2.52617355e-02, -2.66485494e-02,  5.23025915e-02, -6.20203353e-02,
       -6.30565686e-03,  1.90537460e-02, -8.47908780e-02, -3.18325264e-03,
        8.15673918e-02,  3.15989740e-02,  1.26669789e-02, -6.88860118e-02,
        5.05226143e-02, -1.17307929e-02,  1.35211553e-03,  9.08048972e-02,
        5.65620735e-02,  6.38623014e-02,  5.68824820e-03, -2.21508667e-02,
        9.67698731e-03, -3.18021746e-03, -3.81406918e-02, -6.19331794e-03,
       -2.99046524e-02,  7.58718029e-02, -1.09009705e-02,  3.11683980e-03,
        2.62626670e-02, -4.21520993e-02, -4.91012223e-02, -3.53433825e-02,
        1.18838865e-02, -3.81727926e-02,  6.28206739e-03,  9.98245180e-02,
       -1.55796697e-02, -2.14281157e-02,  3.36364061e-02, -1.35120273e-01,
       -3.96473035e-02,  1.59781892e-02, -9.25393682e-03,  1.55161638e-02,
       -5.51356710e-02, -8.38989168e-02, -7.81643391e-02, -5.18350974e-02,
        6.46155030e-02, -5.60104139e-02, -1.12870462e-01, -1.54511224e-32,
       -5.18088192e-02,  5.11351624e-04,  6.85913712e-02,  4.95946594e-02,
       -1.63881741e-02, -6.37725275e-03,  1.05373543e-02, -2.77330372e-02,
        2.10691337e-02, -8.63787811e-03,  8.90394822e-02,  1.00001860e-02,
        1.76549528e-03, -5.51723577e-02,  5.13805524e-02, -3.08592059e-03,
       -1.78555530e-02,  6.74916655e-02, -9.98551175e-02, -7.06799477e-02,
       -5.71440980e-02,  5.78359403e-02, -1.13431424e-01,  5.17467670e-02,
       -4.47607376e-02,  4.57386523e-02,  8.21967721e-02, -1.94483362e-02,
        7.26949275e-02, -1.03210723e-02, -7.13765025e-02, -1.17292469e-02,
       -7.76855741e-03,  3.54502909e-02, -1.19857900e-02,  4.81210053e-02,
        3.22986022e-02,  1.00223795e-02, -4.45140302e-02, -2.74939630e-02,
        4.63612154e-02,  1.12540023e-02,  2.72185784e-02, -8.41479283e-03,
        6.52089762e-03, -1.30446507e-02,  4.68126610e-02,  2.47143004e-02,
       -2.00443976e-02,  2.91587245e-02,  2.98031420e-02,  6.67795539e-02,
       -8.65343884e-02,  7.61257112e-03,  4.36822847e-02, -1.02898166e-01,
       -5.38528040e-02,  8.61857459e-03,  4.77890410e-02, -1.26421437e-01,
        1.18742906e-03,  7.59131312e-02,  5.11817224e-02, -4.85976897e-02,
        2.14014985e-02, -7.58717507e-02, -6.72435910e-02, -3.24591361e-02,
       -6.68453425e-02,  8.80318042e-03,  2.68131681e-02, -1.98948178e-02,
       -1.60693631e-01,  3.74155380e-02,  3.20771709e-02,  1.19849648e-02,
        2.01108642e-02,  7.96646103e-02, -1.77852567e-02,  6.18624082e-03,
        8.12818669e-03,  3.39010209e-02, -7.69744739e-02, -4.57629524e-02,
        4.82442006e-02, -4.51703295e-02,  3.74942645e-02, -3.94024961e-02,
       -7.07456842e-02, -6.83920681e-02, -6.70497268e-02, -1.65324043e-02,
        3.42572890e-02,  3.28121744e-02,  3.73071320e-02, -3.92397830e-08,
        4.95587178e-02,  4.41210531e-02, -2.18693744e-02,  2.65890593e-03,
        7.37099051e-02,  1.97131597e-02, -8.21299925e-02, -6.71027601e-02,
       -3.95517349e-02, -5.80144152e-02,  5.27847447e-02, -5.65863661e-02,
       -1.02416463e-02, -3.33248638e-02,  2.90380176e-02, -4.01263777e-03,
       -3.26377414e-02, -8.58402904e-03, -3.98203656e-02, -8.84262398e-02,
       -2.14854833e-02,  4.09005918e-02, -4.99822944e-03, -3.18792947e-02,
       -5.76394089e-02,  1.14752073e-02, -4.33377996e-02,  4.00907136e-02,
       -1.91305336e-02, -6.44949377e-02, -8.30578879e-02, -9.35628861e-02,
       -3.21952696e-03, -7.68261179e-02,  8.27705581e-03,  1.03555873e-01,
       -3.60205211e-02,  1.45671358e-02,  3.09487488e-02, -2.04965565e-02,
        2.18407847e-02, -1.83681976e-02,  7.59712933e-03,  8.05367678e-02,
       -3.87774780e-02,  1.51757617e-03,  2.34976155e-03,  3.23323570e-02,
        1.51136788e-02, -7.28395283e-02, -1.42816857e-01,  5.73542789e-02,
       -9.34056491e-02, -7.08696172e-02,  5.65155633e-02,  3.55874305e-03,
       -4.35440168e-02, -4.31717522e-02, -3.17653269e-02,  5.09005040e-02,
        8.85736123e-02, -1.59090888e-02, -2.34596850e-03, -1.24097420e-02],
                "k": 3
            }
        }
    }
}

{
  "took" : 60,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 39,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "h_ocid",
        "_id" : "241000592342",
        "_score" : 1.0,
        "_source" : {
          "doc" : "All body interview mean gun realize true. Always gas tell would art. Care analysis particularly choice big face anything. Effort wish field."
        }
      },
      {
        "_index" : "h_ocid",
        "_id" : "241000592344",
        "_score" : 0.99915624,
        "_source" : {
          "doc" : "Success remember everyone plan business change. Song star prepare environmental community. Film ball father."
        }
      },
      {
        "_index" : "h_ocid",
        "_id" : "241000531115",
        "_score" : 0.98105603,
        "_source" : {
          "doc" : "Production kind something leader economy. Wish day Mrs idea will as. Support compare allow still bill relationship impact. Parent quickly sister alone research fish forward."
        }
      }
    ]
  }
}


## Calculating similarity between vector embeddings, leveraging scipy.spatial.distance

In [ ]:
sentence = fake.text()
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

#all-MiniLM-L6-v2
#paraphrase-MiniLM-L6-V2

sentence = "190013322737 190010449419 190070311615 190016461244 190000035517 241000824772 241000824773 190011514705 190011514707 190010898060 190006975795 190011514702 190011465968 190011465969 190010449443 190013322751 190010324119 190010449443 190013322751 190010324119 190013322737 190010449419 190016461244 190000035517 190011465968 190011465969 241000824772 241000824773 190011514705 190011514707 190010898060 190006975795 190011514702 190070311615"
enconder = SentenceTransformer('all-MiniLM-L6-v2',
                              device=torch_device)
plot_embeddings = enconder.encode(sentence,
                                 device=torch_device)

print(torch_device)
print(sentence)
plot_embeddings

In [ ]:
#Measuring the similarity between sentences
from scipy.spatial.distance import cdist as scipy_cdist
#https://medium.com/lett-digital/nlp-efficient-semantic-similarity-search-with-faiss-facebook-ai-similarity-search-and-gpus-274771d0709a
query = ""
col = list(query,sentence)
similarities = scipy_cdist(query, sentence, 'cosine')
similarities = np.around(similarities, decimals=2)
cos_sim = pd.DataFrame(
    data = similarities,
    columns=col,
    index=col)

## Alternative to sentence_transformer - Sentence Embeddings/Enconders/Vectorizing

In [ ]:
#https://medium.com/lett-digital/nlp-efficient-semantic-similarity-search-with-faiss-facebook-ai-similarity-search-and-gpus-274771d0709a
#https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/
#https://huggingface.co/blog/how-to-train-sentence-transformers
#https://pytorch-geometric.readthedocs.io/en/latest/advanced/remote.html
########################################################
#traing my own sentence-transformer (to capture relationship with childs in the branch-tree)
########################################################

## Pipeline to feed index of two vector attributes

In [146]:
ifrom = 5000
ito = 6000 #len(data)

d2 = list(data.items())[ifrom:ito]

pagesize = 40

pagedData = [d2[i:i+pagesize] for i in range(0, len(d2), pagesize)]
#pagedData
#mylist = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
#a = [mylist[i:i+2] for i in range(0, len(mylist), 2)]
#dict(pagedData[2])

listocid_most1000 = df_nested_list.query("ocid_len < 1000").sort_values(by='ocid_len', ascending=False).head(1000)[["key"]].values.tolist()


In [160]:
import random

rand_ocid_key = random.randint(ifrom,ito)
#print(d2[rand_ocid_key])
#print(rand_ocid_key)

numbers = {'first':0,'second':2,'third':3}
print(type(numbers))
print(list(numbers.items())[1])

#fill a list for random access to fetch long list of ocids
listocid_most1000 = df_nested_list.query("ocid_len < 1000").sort_values(by='ocid_len', ascending=False).head(1000)[["key"]].values.tolist()
print(type(listocid_most1000[random.randint(0,len(listocid_most1000))][0]))

<class 'dict'>
('second', 2)
<class 'str'>


In [161]:
import dask
import random


#delayed functions
listToJoinedWordsD = dask.delayed(listToJoinedWords)
stringToVectorD = dask.delayed(stringToVector)
vectorEnhancedDocD = dask.delayed(vectorEnhancedDoc_2)
sendToRedisSearchD = dask.delayed(sendToRedisSearch)

print("Batches: " + str(len(pagedData)))

#to slice the dataset
index = "h_ocid_2"
for chunck in pagedData:
    dchunk = dict(chunck)
    zs = []
    for key in dchunk: 
        #secondary_ocid_key = random.randint(ifrom,ito)
        #secondary_ocid_list = list(data.items())[secondary_ocid_key]
        secondary_ocid_list = data[listocid_most1000[random.randint(0,len(listocid_most1000))][0]]
        listwords1 = listToJoinedWordsD(data[key])
        listwords2 = listToJoinedWordsD(secondary_ocid_list)
        vect1 = stringToVectorD(model, listwords1, torch_device)
        vect2 = stringToVectorD(model, listwords2, torch_device)
        z = vectorEnhancedDocD(key,vect1,vect2,fake,index, data[key], secondary_ocid_list)
        send = sendToRedisSearchD(index, key, z)
        zs.append(send)
        #print(key)
        #send.visualize(filename="transpose_opt.svg", optimize_graph=True)
    #print("Batch of " + str(len(zs)))
    zs = dask.compute(*zs)  # trigger computation in the background


Batches: 25


## Index with two vector attributes and combining query on words string attributes

In [ ]:


#creating index
PUT /h_ocid_2
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "ocid_vec1": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 512,
              "m": 48,
              "ef_search": 256
            }
        }
      },
      "ocid_vec2": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
            "name": "ivf",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "nprobes": 8,
              "nlist": 8
            }
        }
      },
    "doc": { "type": "text" },
    "ocids_1": { "type": "text" },
    "ocids_2": { "type": "text" }
    }
  }
}

GET h_ocid/_search
{
  "query": {
    "match": {
      "doc": "morning" 
    }
  }
}

## Training Data

In [138]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
model = SentenceTransformer('distilbert-base-nli-mean-tokens', device='cpu')


In [143]:
# Transforming/reshaping dataset
dataset = []

dt = list(data.items())
#dt = list(data.items())[0:100]

for idx in dt:
    count = 0
    ls = [int(idx[0])] + idx[1]
    while count < len(ls) - 1:
        #print(ls[count:count+2])
        ie = list(map(str, ls[count:count+2]))
        dataset.append(InputExample(texts=ie, label=0.9))
        count += 1
    


In [144]:
# training
start = time.time()
train_dataloader = DataLoader(dataset, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, output_path='models')

end = time.time()
print('Total time:', (end - start)/60)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/84655 [00:00<?, ?it/s]

Total time: 1020.3794723073642


In [ ]:
# Evaluators
from sentence_transformers import evaluation
sentences1 = ['This list contains the first column', 'With your sentences', 'You want your model to evaluate on']
sentences2 = ['Sentences contains the other column', 'The evaluator matches sentences1[i] with sentences2[i]', 'Compute the cosine similarity and compares it to scores[i]']
scores = [0.3, 0.6, 0.2]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

# ... Your other code to load training data

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)

# Loading Custom SentenceTransformer Models
model = SentenceTransformer('./my/path/to/model/')
#model = SentenceTransformer('http://www.server.com/path/to/model/my_model.zip')
#With the first call, the model is downloaded and stored in the local torch cache-folder (~/.cache/torch/sentence_transformers). In order to work, you must zip all files and subfolders of your model.

In [140]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Tryout dask and GPU: 

In [ ]:
#https://developer.nvidia.com/blog/dask-tutorial-beginners-guide-to-distributed-computing-with-gpus-in-python/
#https://gist.github.com/jacobtomlinson/6f16abb716f50f81a6687bd67efd2f61
#https://docs.rapids.ai/api/cudf/nightly/user_guide/10min.html
import cudf
import dask.config
import getpass
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
cluster = LocalCUDACluster()
client = Client(cluster)

## OpenSearch Index

In [ ]:
#To create a document

PUT h_ocid/_doc/190035525634
{
  "my_vector":[-1.20389678e-01,  2.72174412e-03, -6.14746250e-02, -1.05302865e-02,
       -5.16655110e-02,  7.72307254e-03,  3.15458216e-02,  7.22755492e-02,
        2.15589758e-02, -7.86775500e-02, -1.59886922e-03, -2.68133357e-02,
        3.68731236e-03,  7.16391066e-03, -7.24085867e-02,  1.74891967e-02,
        2.82381382e-02, -1.37097444e-02, -2.31828708e-02, -2.33734529e-02,
       -1.27398884e-02, -9.17124655e-03, -7.73184970e-02, -2.71425229e-02,
       -8.63509402e-02, -6.30821139e-02, -6.35787193e-03,  2.62619145e-02,
       -3.98539193e-02,  1.24874832e-02,  7.34098926e-02,  1.15469005e-02,
        3.56788449e-02,  1.25632919e-02,  9.44149494e-02, -3.87234874e-02,
       -3.44006941e-02,  5.34005724e-02, -6.62265718e-03,  2.72209931e-04,
       -3.49737220e-02,  4.50927997e-03,  6.45675957e-02, -4.14443240e-02,
        7.87169710e-02,  6.30497858e-02, -1.34276254e-02,  1.14531614e-01,
        1.45210531e-02,  7.35251531e-02, -4.69430014e-02,  2.87840609e-02,
        2.23868359e-02,  5.14974967e-02,  2.32304391e-02,  2.66299117e-03,
        2.26085614e-02,  2.97503881e-02, -1.01519041e-02,  5.01786694e-02,
        6.76894262e-02, -1.82775091e-02,  9.40106623e-03, -6.29997551e-02,
        6.66949227e-02,  4.71148826e-02, -4.47102031e-03, -9.99365374e-02,
        1.06219007e-02, -3.04069836e-02,  3.12770111e-03,  3.60862054e-02,
       -4.95276190e-02, -2.70817783e-02,  9.14116390e-03,  9.75828059e-03,
       -1.40666282e-02,  1.54516136e-03,  3.12117357e-02,  5.56087568e-02,
       -3.18004303e-02, -8.66255984e-02, -4.67819646e-02, -2.77464017e-02,
        1.01103811e-02,  6.79823046e-04,  3.79730873e-02,  5.79826832e-02,
        5.70590980e-02,  7.59155825e-02, -3.97106633e-02,  5.67230470e-02,
       -5.46469213e-03,  1.90770792e-04, -5.23847453e-02,  5.62359318e-02,
        1.15142465e-01,  4.22954224e-02, -3.28238308e-02,  8.62674862e-02,
        1.01315388e-02,  6.66365698e-02, -5.94863296e-02,  3.02175898e-02,
        2.20701061e-02,  1.76839437e-02,  1.02713048e-01, -6.70556817e-03,
        3.17404307e-02,  2.52022874e-02,  2.69762333e-02, -6.18687309e-02,
       -5.38737550e-02, -7.97674991e-03,  4.13807333e-02, -6.45891353e-02,
       -7.61196134e-04,  3.74584980e-02,  4.48608026e-02, -1.64294746e-02,
        6.27794936e-02, -6.66400120e-02, -5.21981455e-02, -6.58445284e-02,
       -9.10833627e-02,  3.39489505e-02,  2.87726731e-03,  1.35694834e-33,
       -4.36400324e-02,  3.99701484e-02, -3.42861097e-03,  5.36453053e-02,
       -2.61374805e-02, -1.01138428e-02,  3.35478373e-02,  5.46961790e-03,
        2.11462211e-02, -3.43638510e-02, -7.51503780e-02, -1.04023376e-02,
        3.06058507e-02,  9.93303582e-03, -5.79252429e-02, -9.16027948e-02,
        6.92964718e-02,  1.64650101e-02, -1.43237086e-03,  1.23519627e-02,
        9.04907100e-03, -2.47212891e-02,  1.95832215e-02,  2.22456157e-02,
        5.13252877e-02,  1.14173375e-01, -3.74086350e-02, -3.37765366e-02,
        8.87799188e-02,  3.73926051e-02,  4.34431247e-02, -2.41431128e-02,
       -4.40380387e-02, -5.54073639e-02,  4.79295477e-02, -1.72658060e-02,
        4.24005874e-02, -7.26024061e-02,  1.04519585e-02, -5.92606030e-02,
       -4.36851494e-02,  2.91888770e-02, -6.20751753e-02,  3.30284773e-03,
       -1.32070249e-02, -1.29751358e-02,  8.08862299e-02,  8.90100896e-02,
        9.41451788e-02, -1.76341794e-02, -1.39734417e-01,  1.82850752e-02,
       -1.14711091e-01,  3.27748177e-03, -7.18364641e-02, -9.33197215e-02,
        2.07967218e-02, -5.39144129e-02, -3.43679599e-02, -4.60579060e-02,
        2.20234264e-02,  1.31684601e-01, -1.19883213e-02, -4.05592173e-02,
       -1.44402198e-02, -1.40266940e-01,  5.44718653e-02, -3.50495591e-03,
        4.22157422e-02, -4.05779332e-02, -7.61306956e-02, -2.54742932e-02,
        3.03349155e-03,  2.34607607e-02, -6.00277260e-02, -2.55862344e-02,
        1.82458144e-02, -2.52355039e-02, -6.69198409e-02,  1.06124394e-01,
       -1.49302352e-02,  6.46284968e-02, -4.82585393e-02, -2.87879468e-03,
        1.34466740e-03,  5.29090129e-02, -1.44505873e-02, -1.32282842e-02,
        7.82420039e-02,  8.77197366e-03, -4.26927432e-02, -6.16915291e-03,
        2.63650399e-02,  6.68944791e-02, -2.08375789e-02, -2.30512954e-33,
        1.69839468e-02, -2.27974984e-03, -5.41129187e-02, -3.40281241e-02,
       -5.72250672e-02, -5.34612723e-02,  2.85809226e-02,  3.73635143e-02,
        1.08829048e-02, -1.52059197e-02,  2.09574793e-02, -5.24986051e-02,
        9.08079790e-04, -2.95389742e-02, -1.95298903e-02,  6.83215708e-02,
       -4.59972993e-02,  4.66072373e-02, -2.38574781e-02, -2.78059132e-02,
        4.95987653e-04,  4.67288233e-02, -3.78714129e-02,  6.97557442e-03,
       -6.87732920e-02, -3.59237343e-02,  2.77608410e-02,  1.37071172e-02,
       -1.22102033e-02, -1.96613614e-02, -5.77273360e-03, -4.25215028e-02,
        4.49152812e-02,  6.81878254e-02, -7.20333979e-02,  2.27741729e-02,
        1.66334897e-01,  7.96864368e-03,  5.17632253e-03, -6.11172104e-03,
        6.75259233e-02,  5.74311912e-02, -2.49375235e-02,  1.36555746e-01,
       -3.11049558e-02,  5.77290803e-02, -3.67946997e-02, -7.49284178e-02,
        1.77905764e-02, -8.65405519e-03,  4.05460633e-02,  9.17671062e-03,
       -8.31936207e-03,  5.80609068e-02,  4.88902852e-02,  2.04396807e-02,
        9.46582034e-02,  7.23294616e-02,  1.72563791e-02, -2.11634133e-02,
        1.04809087e-03,  5.16194887e-02,  3.85425761e-02,  5.57987876e-02,
        6.20483570e-02, -6.73913350e-03, -2.46340618e-03, -2.58684042e-03,
       -1.22921960e-02, -2.43539959e-02,  1.33881136e-03, -1.84672289e-02,
        3.37614299e-04,  6.64041117e-02, -2.79421341e-02, -3.07002533e-02,
       -1.21695595e-02,  2.64683478e-02, -2.17593834e-02,  6.01380132e-02,
        1.07274950e-01,  6.03950843e-02,  3.84047776e-02, -8.97937045e-02,
       -3.80981900e-03, -1.51200807e-02,  1.45865947e-01,  1.06363036e-01,
       -6.73792586e-02, -8.03166255e-02, -1.19250976e-02, -6.77438919e-03,
       -8.11464861e-02, -9.57522839e-02, -7.88728669e-02, -1.74856023e-08,
        2.69645974e-02, -2.23521236e-02,  1.00894831e-02, -6.33337051e-02,
        7.04772547e-02, -1.77828111e-02, -1.95563566e-02,  1.97855607e-02,
       -1.22462017e-02,  1.30608003e-03,  5.10606021e-02, -8.95731803e-03,
       -9.13310200e-02,  3.72961499e-02,  4.46502790e-02, -8.78806487e-02,
       -1.27439812e-01,  7.57507933e-03, -6.55077223e-04,  3.86025496e-02,
        4.33169259e-03,  3.88793275e-02, -6.59813732e-03, -2.77327932e-02,
       -7.51499226e-03, -3.88742760e-02,  3.53232063e-02,  9.37039107e-02,
       -7.52042383e-02, -2.83239726e-02,  5.31701781e-02,  5.22215292e-02,
       -9.65833217e-02, -4.22533266e-02, -9.24448594e-02,  5.23625128e-02,
       -1.10741116e-01,  3.07659619e-02,  2.10411660e-03, -4.26713154e-02,
        7.56452158e-02, -5.09953201e-02, -5.77574447e-02, -1.18250791e-02,
       -3.67471087e-03, -1.63946617e-02, -3.60019766e-02, -7.99591318e-02,
       -7.84831643e-02, -2.87576970e-02, -9.68581438e-02, -3.41796242e-02,
        6.91488236e-02, -5.05279303e-02, -3.00676227e-02, -3.87922265e-02,
       -3.59750465e-02,  1.33169601e-02,  6.95136283e-03,  6.12755083e-02,
        6.15849793e-02,  1.00964986e-01, -5.85744306e-02, -4.97348085e-02],
  "doc": "Necessary no expert. Feeling fire teach recognize. Actually race goal example clearly though ten morning.\nCertainly week success many.",
  "ocids": [190070311615]
}

In [ ]:
#To query in a vector field
#https://opensearch.org/docs/latest/search-plugins/knn/filter-search-knn/

GET h_ocid/_search
{
  "query": {
    "match": {
      "doc": "data" 
    }
  }
}

GET h_ocid/_search
{
  "query": {
        "match_all": {}
    
  }
}

GET h_ocid/_count

#After vectorizing the value '190070311615' and persisting that in the Opoensearch
#the the part of that value was also vectorize '703116' and used to perform a query by similarity
#resulting in a similarity of 80% 
#    "max_score" : 0.813945,
#    "hits" : [
#      {
#        "_index" : "h_ocid",
#        "_id" : "190035525634",
#        "_score" : 0.813945,


#if I change the vectorized query valeu to get closer to the origin value, the similarity increases
#For instance using the value vector from the value '9007031161', the similarity:
#    "max_score" : 0.8838218,
#    "hits" : [
#      {
#        "_index" : "h_ocid",
#        "_id" : "190035525634",
#        "_score" : 0.8838218,

#now the vector from '90070311615'
#    "max_score" : 0.8995489,
#    "hits" : [
#      {
#        "_index" : "h_ocid",
#        "_id" : "190035525634",
#        "_score" : 0.8995489,

#origin value            '190070311615'
#Scores
##query with vector from '703116'       => 0.813945%
##query with vector from '9007031161'   => 0.8838218%
##query with vector from '90070311615'  => 0.8995489,%

#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('all-MiniLM-L6-v2')

#sentences = ['703116']
#sentence_embeddings = model.encode(sentences)


POST /h_ocid/_search
{
    "size": 3,
    "query": {
        "knn": {
            "my_vector": {
                "vector": [-5.55934869e-02,  1.29339332e-02, -1.28656656e-01,
         3.58514041e-02, -7.30582029e-02, -1.04542840e-02,
        -1.87872387e-02,  4.82300706e-02, -4.37816568e-02,
        -2.61467788e-02, -1.74138118e-02, -8.84978771e-02,
        -2.92600710e-02,  5.91269368e-03, -7.54115954e-02,
        -3.11078914e-02, -5.58171375e-03, -2.99040321e-03,
         1.07176090e-02, -2.90980283e-02, -6.50314763e-02,
         1.21966470e-02, -6.87315539e-02,  2.54249442e-02,
        -3.37093808e-02, -1.30437789e-02,  4.46806625e-02,
         3.10143568e-02,  5.84513135e-03, -5.51805571e-02,
         5.48519902e-02,  4.23047580e-02,  8.35865065e-02,
         2.28181537e-02,  5.01376130e-02, -8.67037196e-03,
        -2.98762992e-02, -6.64138049e-02, -3.22493240e-02,
         6.72521675e-03, -5.22062369e-02,  9.13577341e-03,
         7.27580711e-02, -1.40315322e-02,  1.35981347e-02,
         7.07495809e-02, -1.70226134e-02,  5.50088398e-02,
         2.78230868e-02,  1.95499770e-02, -7.90448934e-02,
         2.70435344e-02, -2.53945719e-02,  9.07937363e-02,
         4.06730138e-02, -1.86187553e-03,  1.87429716e-03,
        -4.58768234e-02,  3.59620973e-02,  6.44419119e-02,
         3.13751660e-02, -6.08937480e-02, -1.15714269e-02,
        -6.29759878e-02,  2.82588750e-02, -1.60676930e-02,
        -1.88103386e-05, -4.51191328e-02,  1.13825370e-02,
        -4.57077995e-02,  1.75623987e-02,  8.34060311e-02,
         9.56142507e-03,  9.62366816e-03,  5.45912683e-02,
        -2.96479673e-04, -4.57575880e-02,  1.86456330e-02,
         5.70671894e-02,  1.32700913e-02,  1.41065742e-03,
        -3.36810052e-02, -4.55485843e-02, -2.05124673e-02,
         3.61745395e-02, -1.07600726e-02,  4.42223176e-02,
         8.36957097e-02,  6.41425699e-02,  5.01059666e-02,
        -7.39199715e-03,  7.55859986e-02,  1.14409076e-02,
        -1.85822193e-02, -6.37974665e-02,  3.94727774e-02,
         9.17915627e-02,  1.93362478e-02, -8.18246976e-03,
         1.04292959e-01,  3.65625843e-02,  9.00046974e-02,
        -1.19806916e-01,  2.83718910e-02,  2.42052712e-02,
         4.20576185e-02,  1.07219130e-01,  4.16781828e-02,
         3.47371958e-02, -5.74513618e-03,  2.76094582e-02,
        -3.99074443e-02,  6.08235449e-02,  3.71283805e-03,
         3.92920338e-02, -4.04980555e-02,  5.30654266e-02,
        -3.52560282e-02, -9.02287848e-03, -2.23532431e-02,
         5.92207313e-02, -5.81754446e-02,  5.60741536e-02,
        -4.28607389e-02, -4.11018543e-02,  1.25633366e-02,
         2.23901123e-02, -1.99785068e-34, -2.33324245e-02,
         2.30608843e-02,  1.11321468e-04,  2.43394542e-02,
        -2.26294566e-02, -7.27064535e-02,  2.12875661e-02,
        -2.52368227e-02,  2.91248243e-02, -5.82720116e-02,
        -5.39482124e-02,  1.29479272e-02,  3.26254778e-02,
        -3.31934169e-02, -5.62832914e-02, -7.42130354e-02,
         6.69471547e-02,  1.25499405e-02, -9.06537345e-04,
         3.41138206e-02, -3.43717076e-02,  4.38974064e-04,
         3.22768129e-02,  7.41799772e-02,  6.31709024e-02,
         7.38151670e-02, -7.61171430e-02, -2.62498111e-02,
         1.23777211e-01,  3.09303105e-02,  8.05376545e-02,
        -4.18597236e-02,  1.27534568e-02, -2.27116235e-02,
         1.40171852e-02, -5.28978221e-02, -2.75882129e-02,
        -8.37005228e-02,  8.43771454e-03,  3.77546530e-03,
         1.43352263e-02,  2.89744567e-02, -1.47927850e-02,
         2.46193283e-03, -3.97293158e-02,  2.29029730e-02,
         9.80481282e-02,  7.68947452e-02,  5.60916178e-02,
        -1.66792441e-02, -1.02694832e-01,  1.32698156e-02,
        -1.31249696e-01, -3.19351978e-03, -8.13418329e-02,
        -3.58230248e-02,  1.09379599e-02, -5.60984835e-02,
        -6.31018775e-03, -5.56376055e-02,  2.02158894e-02,
         1.40332133e-01, -2.35382318e-02, -3.96524444e-02,
        -1.36594959e-02, -1.12907805e-01,  2.05080025e-02,
         2.66920263e-03,  8.22720397e-03, -3.69946193e-03,
        -8.64256397e-02, -3.07945665e-02,  4.36583720e-02,
         3.20863314e-02, -6.03489727e-02, -6.33564517e-02,
         6.22521853e-03, -8.30158144e-02, -8.64129588e-02,
         1.20627329e-01,  7.96278566e-03,  8.27060863e-02,
        -6.50874376e-02, -1.32614397e-03, -2.07985621e-02,
        -2.74772458e-02, -2.80393064e-02, -6.01352938e-02,
         6.19861484e-02, -2.69788206e-02, -6.24600947e-02,
        -1.17943445e-02,  6.96161985e-02,  4.59036976e-02,
        -3.72970477e-02,  2.65866677e-34,  2.69848239e-02,
         1.34771438e-02, -3.84058692e-02,  3.03699970e-02,
        -8.36950243e-02, -1.38431443e-02,  1.54578369e-02,
         2.93550156e-02, -2.86995601e-02, -5.20194247e-02,
         3.94120365e-02, -3.21829468e-02,  7.66378120e-02,
        -5.03205694e-03, -9.23020486e-03,  5.70712797e-03,
        -1.07023835e-01,  7.58812055e-02, -6.90636933e-02,
        -3.91102657e-02, -3.56564708e-02,  5.29545620e-02,
        -5.81365302e-02,  2.00890973e-02, -1.67104490e-02,
        -2.06963555e-03,  2.56808996e-02,  2.04513278e-02,
         3.07323132e-02, -1.96885900e-03,  8.63447133e-03,
        -4.39941436e-02,  6.94412664e-02,  7.50013888e-02,
        -5.65401390e-02, -1.75816752e-02,  1.45473137e-01,
        -1.36715397e-02,  5.50750792e-02, -2.74170786e-02,
         1.02814846e-01,  3.26334983e-02,  7.72588840e-03,
         8.22911412e-02, -2.98278071e-02,  3.24156284e-02,
         3.35139148e-02, -3.94643238e-03,  2.10322496e-02,
        -6.29381463e-02,  9.79490206e-03,  5.10655418e-02,
        -4.57685702e-02,  7.92087317e-02,  3.51892225e-02,
         2.80038510e-02,  2.63893213e-02,  5.45069166e-02,
         8.85692015e-02, -4.34388407e-02,  3.44000198e-02,
         4.58497070e-02,  8.56760293e-02,  5.33634014e-02,
         5.61020859e-02, -4.91176657e-02, -5.78193292e-02,
         3.36540118e-02, -3.09017785e-02, -5.33038564e-03,
        -2.89380387e-03, -4.07708883e-02, -3.80229275e-03,
         1.06949739e-01, -1.45148449e-02, -3.48845012e-02,
        -1.36854080e-02, -6.07296405e-03, -4.90808450e-02,
         9.11619142e-02,  4.83669527e-02,  7.06447363e-02,
         3.05592939e-02, -6.65579513e-02, -7.08973454e-03,
         2.01108698e-02,  1.82297289e-01,  6.29160404e-02,
        -5.57216257e-02, -7.88268223e-02, -1.72342137e-02,
         1.59093121e-04, -4.79997061e-02, -5.43109141e-02,
        -2.82672327e-02, -1.34649030e-08,  4.12397236e-02,
        -5.70580438e-02,  6.40483107e-03, -6.80006444e-02,
         8.43641311e-02, -2.04112679e-02, -2.85852794e-02,
         4.10693735e-02, -1.23678735e-02, -1.68727618e-03,
         9.13570523e-02, -2.70200018e-02, -9.17480290e-02,
         4.31625769e-02,  3.00727673e-02, -2.51629576e-02,
        -1.08771473e-01, -3.82549986e-02, -1.23247504e-02,
        -1.10009534e-03,  5.78783080e-03,  1.12010296e-02,
         2.66833957e-02,  4.80271503e-03, -4.57415096e-02,
        -4.35016789e-02, -1.21338491e-03,  1.44444987e-01,
        -1.05270803e-01, -1.20409295e-01,  2.80432776e-02,
         8.80384818e-02, -2.89523583e-02, -4.25313041e-02,
        -6.03330620e-02,  5.65446615e-02, -7.69104064e-02,
         6.05587177e-02,  4.17464897e-02,  5.11286221e-02,
         4.88871597e-02, -2.49953894e-03, -3.02036237e-02,
        -1.73257533e-02, -7.37717897e-02, -2.31036711e-02,
         1.14578493e-02, -2.98055932e-02, -5.41873537e-02,
        -1.52390003e-02, -4.03163247e-02, -2.64128149e-02,
         6.04060702e-02, -5.49606495e-02,  4.95908596e-02,
         2.07468364e-02, -6.59692511e-02,  8.96449666e-03,
         7.11396616e-03,  4.82575409e-02,  3.36451903e-02,
         4.24357280e-02, -5.16567416e-02, -3.59659493e-02],
                "k": 3
            }
        }
    }
}


#zsh: segmentation fault  python
#https://github.com/UKPLab/sentence-transformers/issues/1319

In [ ]:
#new validation with a vectorized list of 15 ocid elements
# Milvus: A Vector Database For Unstructured Data Processing  https://www.youtube.com/watch?v=_x8RTSaiT50
#Yet another CAP Theorem to consider:
##Cost
##Accuracy
##Performance

#to encode: 241000641890 241001116483 241000012834 241000012837 241000012838 241001116487 241000012839 241000692457 241000513193 241000513548 241000513501 241000771666 241000673529 241000673531 241000673533


#origin value            *to encode
#Scores
##query with vector from '241000012839'                => 0.67%
##query with vector from '241001116487 241000012839'   => 0.7665135%
##query with vector from '241000673531 241000673533'  => 0.8157501%



PUT h_ocid/_doc/241000513543
{
  "my_vector":[ 3.28153595e-02, -3.41466479e-02, -8.47388580e-02, -4.16152887e-02,
       -9.88707989e-02, -8.80153403e-02,  1.47235468e-02,  5.66860624e-02,
        1.74197350e-02,  2.74440534e-02, -1.15475012e-02,  7.30296271e-03,
        7.27552623e-02,  2.27037594e-02, -5.21193855e-02, -4.46592793e-02,
       -1.13715596e-01, -1.05187140e-01, -1.83852557e-02, -4.16362584e-02,
        5.71160018e-02,  1.57149304e-02, -5.72661385e-02, -5.20927273e-02,
        4.13295962e-02,  1.19665861e-02,  3.43143009e-03,  6.66022971e-02,
       -3.79833137e-03, -5.45456149e-02,  7.70671107e-03,  6.24316521e-02,
        1.37631178e-01, -3.37395770e-03,  3.86067620e-03,  4.89162281e-02,
       -3.84795968e-03, -4.38505858e-02,  3.45634185e-02,  2.64659803e-02,
       -3.04384297e-03, -3.98321487e-02,  8.07543993e-02,  7.12549761e-02,
        1.28568225e-02, -6.27680942e-02, -6.67439550e-02,  7.81251397e-03,
        1.24188373e-02, -4.30286936e-02, -2.35252529e-02,  1.36317179e-01,
       -2.01741955e-03,  5.62207401e-02, -1.26708196e-02, -9.48218256e-02,
       -2.99107451e-02, -9.48386639e-03,  6.19013747e-03,  2.85291933e-02,
       -9.83619038e-03,  7.80017152e-02, -1.44176362e-02, -5.61322086e-02,
        1.69622805e-02,  5.55288605e-03, -5.29111624e-02, -9.98622850e-02,
       -7.19514256e-03,  1.52804879e-02, -2.96257669e-04,  1.86046511e-02,
        2.52393093e-02,  1.29172295e-01,  2.92536840e-02,  3.62881795e-02,
       -7.55011663e-02, -3.68574187e-02,  2.36332938e-02,  3.37606892e-02,
       -5.65881096e-02, -4.49767411e-02,  2.92362701e-02,  7.16865808e-02,
       -2.80204869e-04,  3.49182934e-02, -1.25410734e-02,  6.34524971e-02,
        2.73405351e-02,  1.80085395e-02, -6.06180131e-02, -9.36254952e-03,
        9.00700316e-03,  1.98979340e-02, -7.18510821e-02,  8.48854631e-02,
        9.71342996e-02, -3.75393182e-02, -7.11708665e-02,  7.39181265e-02,
       -3.68691012e-02,  1.75390728e-02, -3.03823147e-02,  9.45670623e-03,
       -1.32463621e-02, -2.29081810e-02,  4.84999754e-02,  2.36461461e-02,
        1.18537741e-02, -3.89019810e-02, -1.43566094e-02, -7.80355632e-02,
       -1.80934882e-03, -4.18551750e-02,  5.99900447e-02, -6.09665252e-02,
       -8.99265856e-02,  4.61979695e-02, -2.74712406e-03,  2.32046805e-02,
        7.31104240e-02,  1.06147854e-02, -5.48059754e-02,  6.00719787e-02,
       -7.35455230e-02, -1.20298341e-02, -2.64275391e-02,  5.60988700e-33,
       -1.89412422e-02,  4.54984568e-02,  6.07713200e-02,  1.59784593e-02,
       -2.15268750e-02,  2.60489788e-02, -1.07301690e-01, -9.81915393e-04,
       -5.48116714e-02,  3.12655494e-02, -2.45416798e-02, -1.56789850e-02,
        7.93487206e-02,  3.18066999e-02,  5.66821545e-02, -5.62461466e-03,
        4.44141440e-02,  1.73580851e-02, -6.59264177e-02, -8.54833201e-02,
        3.19693014e-02,  1.28755942e-02, -8.00970290e-03, -3.41794267e-02,
        7.16968030e-02,  9.59354490e-02, -9.60770063e-03, -5.82833625e-02,
        8.18075165e-02,  5.07496037e-02,  5.22848044e-04,  4.30807024e-02,
       -1.07665680e-01, -2.79153921e-02,  6.41183779e-02, -8.42583179e-02,
        8.56107753e-03,  1.34375191e-03, -3.66933011e-02,  9.60547104e-03,
        5.44060534e-03,  3.62604158e-03, -3.10693178e-02, -6.57723099e-02,
        5.37222475e-02, -1.90099869e-02,  4.88991179e-02, -6.05825558e-02,
        2.78717577e-02,  5.78115433e-02, -6.29749894e-02,  1.75839849e-03,
        9.27953981e-03,  4.50901175e-03,  9.95791517e-03, -7.71615356e-02,
        5.67157231e-02, -9.02310479e-03, -1.75320394e-02,  8.24971050e-02,
        6.96664350e-03,  9.90046561e-02,  2.72789970e-02, -5.11784628e-02,
       -2.04023328e-02, -1.28421439e-02, -5.40333986e-02,  3.14665725e-03,
       -4.31403965e-02,  7.65616819e-02, -6.59620613e-02, -9.04612150e-03,
        6.07407540e-02, -1.56279970e-02, -1.06629543e-01, -2.96403076e-02,
       -3.56273609e-03, -2.07137503e-02, -4.27166112e-02,  5.16284741e-02,
       -2.41228882e-02, -1.24369171e-02,  2.75277663e-02, -1.33146271e-01,
       -2.24543530e-02,  1.40189258e-02, -6.18116446e-02, -5.07578580e-03,
       -3.29517946e-02, -4.18736078e-02, -7.02608302e-02, -4.95486520e-02,
        8.94754454e-02, -6.66067004e-02, -1.36982828e-01, -8.53200028e-33,
       -8.10767934e-02,  2.93114781e-02, -2.16444246e-02,  2.41982006e-02,
       -4.30865437e-02, -5.90079576e-02,  2.56787241e-02,  5.72415367e-02,
        3.58147174e-02,  2.38666181e-02,  5.67477681e-02,  1.26685277e-02,
        3.32577266e-02, -6.60909265e-02,  6.15799576e-02, -2.46481430e-02,
       -1.03958379e-02,  7.76944607e-02, -7.93506280e-02, -1.83143318e-02,
       -3.15140150e-02,  6.51716664e-02, -6.85124025e-02,  7.92408586e-02,
       -1.97569262e-02,  7.68127516e-02,  3.48252170e-02, -4.00224850e-02,
        4.61205356e-02,  3.51404096e-03, -4.02688272e-02, -4.27964181e-02,
       -1.80999804e-02,  3.04949991e-02,  9.28452797e-03, -2.45331936e-02,
        7.57408142e-02,  6.39296928e-03, -1.40326964e-02, -4.01666053e-02,
        5.39453886e-02,  2.09629140e-03,  2.13507190e-02,  4.60130833e-02,
        3.64584029e-02, -3.09575181e-02,  5.90832904e-02,  3.03452346e-03,
       -2.29820795e-02, -6.69827266e-03,  2.36277264e-02,  2.00597253e-02,
       -7.89630041e-02,  3.59871536e-02,  7.77558833e-02, -4.57018092e-02,
       -2.12345235e-02,  9.73579213e-02,  5.80146648e-02, -9.88071784e-02,
       -1.08639328e-02,  2.06266325e-02,  4.98854443e-02,  9.75805428e-03,
        3.41662504e-02, -6.70801848e-02, -9.54605862e-02, -3.47114690e-02,
       -9.20493528e-02, -1.42945135e-02,  2.83257179e-02,  5.14320005e-03,
       -1.43993035e-01,  5.79386912e-02,  1.69843696e-02,  1.02426670e-03,
       -8.26895609e-03,  8.09477195e-02, -4.89218161e-02,  3.89687940e-02,
        3.81102376e-02,  4.03031595e-02, -4.69297357e-02, -1.81601457e-02,
        3.83547731e-02, -4.44130413e-02,  4.75153998e-02,  9.95403808e-03,
       -7.77298734e-02, -7.16140792e-02, -5.42956926e-02,  2.27781516e-02,
        1.27427431e-03,  1.36919562e-02,  5.42310700e-02, -3.39731194e-08,
        3.99615765e-02, -2.91027296e-02, -1.81021877e-02, -5.95621485e-03,
        7.87300020e-02, -1.32476473e-02, -4.50620651e-02, -3.09800226e-02,
       -6.87971013e-03, -4.48847152e-02,  6.00927956e-02, -3.90625782e-02,
       -9.28494930e-02, -2.21725814e-02,  4.57763560e-02, -3.29370610e-03,
       -2.50882786e-02, -3.04552130e-02, -4.07676883e-02, -6.01923019e-02,
        2.52013765e-02,  3.54009983e-03, -5.40744066e-02, -5.63937537e-02,
       -5.91749698e-02, -2.05522906e-02, -6.63530082e-02,  8.60971138e-02,
       -5.02235815e-02, -8.52437168e-02, -3.38159837e-02, -3.16844694e-02,
       -1.60525329e-02, -1.17977306e-01, -2.57707965e-02,  9.63074639e-02,
        2.75786594e-03,  4.92044911e-02,  4.38458435e-02, -6.83320165e-02,
       -2.05658432e-02, -3.73967690e-03, -2.96943393e-02,  5.10239564e-02,
        5.65446273e-04, -3.58609296e-02,  6.36776024e-03,  1.76308677e-03,
       -1.66940894e-02, -1.20284714e-01, -9.67940092e-02,  6.38389140e-02,
       -4.11676578e-02, -8.52283984e-02,  4.74898480e-02, -2.91564711e-03,
       -6.92062899e-02, -5.33880517e-02, -3.72043736e-02,  1.01547033e-01,
        6.73423856e-02,  2.33578496e-02,  1.05107790e-02, -3.98011506e-02],
  "doc": "Alone tell past everyone. Player history hard table tough million onto.\nHealth operation health voice. Lawyer best out anything artist.",
  "ocids": [241000641890, 241001116483, 241000012834, 241000012837, 241000012838, 241001116487, 241000012839, 241000692457, 241000513193, 241000513548, 241000513501, 241000771666, 241000673529, 241000673531, 241000673533]
}

In [ ]:


settings = {
  "settings": {
    "elastiknn": True,
    "number_of_shards": 1,
    "number_of_replicas": 0
  }
}

mapping = {
  "dynamic": False,
  "properties": {
    "asin": { "type": "keyword" },
    "imVecElastiknn": {
      "type": "elastiknn_dense_float_vector",
      "elastiknn": {
        "dims": 768,
        "model": "lsh",
        "similarity": "angular",
        "L": 60,
        "k": 3
      }
    },
    "imVecXpack": {
      "type": "dense_vector",
      "dims": 768
    },
    "title": { "type": "text" },
    "description": { "type": "text" },
    "price": { "type": "float" },
    "imUrl": { "type": "text" }
  }
}

if not es.indices.exists(index):
  es.indices.create(index, settings)
  es.indices.put_mapping(mapping, index)
es.indices.get_mapping(index)

In [ ]:
#OpenSearch queries
PUT /myindex
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "knn_vector",
        "dimension": 2
      }
    }
  }
}

PUT /h_ocid
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "knn_vector",
        "dimension": 768
      },
    "seq_ocid": { "type": "text" }
    }
  }
}

PUT /myindex/_doc/1
{
  "my_vector": [1.5, 2.5]
}

PUT /myindex/_doc/2
{
  "my_vector": [2.5, 3.5]
} 

POST /myindex/_search
{
  "size": 2,
  "query": {
    "knn": {
      "my_vector": {
        "vector": [1, 1.5],
        "k": 5
      }
    }
  }
}

#combine the knn query clause with other query clauses
POST /myindex/_search
{
  "size": 5,
  "query": {
    "bool": {
      "must": {
        "knn": {
          "my_vector": {
            "vector": [3, 4],
            "k": 5
          }
        }
      },
      "filter": {
        "range": {
          "price": {
            "lt": 15
          }
        }
      }
    }
  }
}

In [ ]:
# Opensearch queries



GET /_cat/indices

DELETE /h_ocid

PUT /h_ocid
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 256,
              "m": 48
            }
        }
      },
    "doc": { "type": "text" },
    "ocids": { "type": "text" }
    }
  }
}

GET _search
{
  "query": {
    "match_all": {}
  }
}

GET h_ocid/_count


GET h_ocid/_search
{
  "query": {
    "match": {
      "_id": "241000592342" 
    }
  }
}

GET h_ocid/_search
{
  "query": {
    "match": {
      "_id": "241000513543" 
    }
  }
}


GET h_ocid/_search
{
  "query": {
    "match": {
      "_id": "241000592342" 
    }
  }
}

POST /h_ocid/_search
{
    "size": 3,
    "_source": [ "doc", "_id", "ocids" ],
    "query": {
        "knn": {
            "my_vector": {
                "vector": [ 3.63255702e-02, -3.63128819e-02, -7.54057914e-02, -3.16446498e-02,
        -9.89066511e-02, -6.94269463e-02,  3.34163308e-02,  5.13368547e-02,
         2.91657504e-02,  1.64758600e-02, -2.04364378e-02, -4.98858653e-03,
         5.59058338e-02,  2.77740750e-02, -3.38620767e-02, -3.22797187e-02,
        -8.72896910e-02, -1.27544448e-01, -1.21594192e-02, -2.03348901e-02,
         5.81924245e-02,  1.41064702e-02, -4.10989746e-02, -4.84944955e-02,
         1.25774425e-02,  1.41558312e-02, -6.62191585e-03,  6.62363991e-02,
        -1.25964405e-02, -6.91132024e-02, -6.82661356e-03,  8.10910463e-02,
         9.30521116e-02,  1.65924500e-03, -1.26757044e-02,  1.38554508e-02,
        -9.00699757e-03, -1.75885335e-02,  4.73585203e-02,  1.16737206e-02,
         2.93568578e-02, -4.51500863e-02,  1.09885834e-01,  7.53370225e-02,
        -1.25225473e-04, -3.52009386e-02, -6.26274720e-02,  2.49078982e-02,
        -1.00564817e-02, -3.00759319e-02,  1.40724157e-03,  1.19276546e-01,
        -2.23196149e-02,  5.63190430e-02, -1.98846054e-03, -1.03240885e-01,
        -1.31866382e-02, -2.83154007e-02,  1.95111800e-02,  6.32438660e-02,
         3.28175575e-02,  6.61696494e-02, -1.09526077e-02, -4.51404490e-02,
         4.29810118e-03, -6.12927042e-03, -2.91464310e-02, -1.11973047e-01,
        -2.09968928e-02, -5.77262510e-03,  2.25738455e-02,  3.67104374e-02,
         5.34984805e-02,  7.37569407e-02,  1.45483874e-02,  1.61951110e-02,
        -3.85679938e-02, -5.36367707e-02,  1.59074683e-02,  3.74365225e-02,
        -3.33444253e-02, -3.57522108e-02, -9.47802886e-03,  3.21278870e-02,
        -3.20120193e-02,  3.97224315e-02,  6.90222485e-03,  5.17134778e-02,
         1.39812119e-02,  9.96654481e-03, -6.08248115e-02,  3.23595256e-02,
         3.32815535e-02,  4.55344729e-02, -8.51950273e-02,  9.28852111e-02,
         8.53299499e-02, -1.51685495e-02, -5.13066128e-02,  6.26358762e-02,
        -1.32279266e-02,  5.17625958e-02, -2.90367845e-02,  1.51292712e-03,
        -2.06963010e-02, -3.74151655e-02,  4.42146324e-02,  1.37613295e-02,
         1.47293834e-02, -3.54372524e-02,  9.40942671e-03, -9.56384987e-02,
         3.21595483e-02,  2.32631378e-02,  5.58130257e-02, -4.03990224e-02,
        -5.58147579e-02,  4.49959636e-02,  1.05793020e-02,  3.11322194e-02,
         1.07754469e-01,  3.90361436e-02, -5.82668856e-02,  4.54272032e-02,
        -8.48090053e-02,  1.04751764e-02, -1.63952541e-02,  4.36108551e-33,
        -2.48437393e-02, -6.24772953e-03,  7.97562376e-02,  3.02414019e-02,
        -3.77275869e-02,  1.04305316e-02, -1.10663332e-01,  1.78453512e-02,
        -4.36298475e-02,  4.66400906e-02, -3.57871205e-02, -1.63143836e-02,
         5.17589897e-02,  2.83818357e-02,  5.44459820e-02,  1.74056608e-02,
         5.17556556e-02,  2.57268399e-02, -6.21175915e-02, -7.57692456e-02,
         4.64512184e-02,  4.46902737e-02, -2.11006962e-03, -4.19100784e-02,
         8.55025575e-02,  5.77421635e-02, -1.16868066e-02, -6.56843260e-02,
         8.52573812e-02,  4.18928638e-02,  1.49885835e-02,  4.68509011e-02,
        -6.91203922e-02,  3.10928607e-03,  3.08177471e-02, -5.98321185e-02,
         5.44269010e-02, -1.51289329e-02, -2.01584492e-02,  8.16028751e-03,
        -9.24962107e-03,  1.66458264e-02, -3.14567015e-02, -7.51507655e-02,
         4.66259383e-02, -4.62953374e-03,  8.10802355e-02, -4.53542806e-02,
         4.20669578e-02,  4.47624512e-02, -8.80535096e-02,  2.30463035e-02,
        -6.76790113e-03,  1.41911628e-02,  4.99189692e-03, -8.63917395e-02,
         6.07668683e-02, -3.19136977e-02, -3.35042328e-02,  1.14145286e-01,
        -4.81523434e-03,  1.04983792e-01,  1.99269429e-02, -3.64078097e-02,
        -1.74554568e-02, -3.74761149e-02, -4.63602282e-02, -1.24903545e-02,
        -3.24301422e-02,  6.40397966e-02, -4.65193726e-02, -3.64997797e-02,
         7.13751316e-02, -1.94848015e-03, -7.50759766e-02, -5.11382967e-02,
        -2.37912731e-03,  6.66293781e-03, -4.92489412e-02,  7.04353154e-02,
        -5.31207360e-02,  5.94947068e-03,  9.06533655e-03, -1.30806029e-01,
         5.93469711e-03,  5.83826448e-04, -6.54386505e-02,  5.70948003e-03,
        -2.26079933e-02, -4.09635566e-02, -9.36654508e-02, -4.68591005e-02,
         7.85738081e-02, -6.18832223e-02, -1.49612427e-01, -7.57866187e-33,
        -9.55005437e-02,  4.06575836e-02, -5.16766757e-02,  7.85988383e-03,
        -6.04318678e-02, -7.43322298e-02,  4.19110022e-02,  6.98147863e-02,
         3.47498953e-02,  4.06702049e-02,  5.63311763e-02, -2.84955502e-02,
         4.76909243e-02, -3.58998366e-02,  3.63287479e-02, -1.77358780e-02,
         1.13555938e-02,  5.45977429e-02, -7.38854557e-02, -3.13994940e-03,
        -1.89735144e-02,  6.19610846e-02, -7.34625533e-02,  9.31313783e-02,
        -2.37276927e-02,  6.86981604e-02,  4.09363732e-02, -1.10821784e-01,
         5.98575734e-03,  2.97835190e-02, -3.77468914e-02, -9.41876546e-02,
        -1.95738673e-02,  8.78348108e-03, -1.96203757e-02, -4.80398387e-02,
         9.33861807e-02, -2.58936565e-02, -4.15612794e-02, -1.71279740e-02,
         5.33629172e-02,  1.41055109e-02,  4.29882780e-02,  4.86377217e-02,
         1.96516421e-02,  9.27241426e-03,  6.85240477e-02, -2.52116621e-02,
        -3.02328281e-02, -4.40428145e-02,  1.73444916e-02,  1.38086956e-02,
        -9.58904028e-02,  3.33534181e-02,  3.77789214e-02, -4.12489399e-02,
         4.32637893e-03,  8.70631039e-02,  2.39823181e-02, -7.33574331e-02,
         2.18269750e-02,  3.51227671e-02,  5.89158274e-02,  4.46308637e-03,
         1.12120789e-02, -7.53286555e-02, -6.30009025e-02, -3.56468149e-02,
        -9.35283154e-02,  9.91627946e-03, -6.94584288e-03,  1.89228505e-02,
        -1.58963218e-01,  2.85670832e-02,  2.09364537e-02,  2.28818282e-02,
        -1.65012311e-02,  5.96850030e-02, -4.22890931e-02,  2.70378739e-02,
         3.20426114e-02,  3.89549062e-02, -2.53528710e-02, -7.59811327e-02,
         1.11147631e-02, -4.18808982e-02,  6.87555373e-02,  1.88958738e-02,
        -6.51089475e-02, -7.10437745e-02, -5.33065386e-02,  2.19933167e-02,
        -1.54782105e-02, -5.04482053e-02,  4.50361706e-02, -3.20942952e-08,
         3.07912808e-02, -9.12830513e-03, -2.21148022e-02, -5.68969315e-03,
         7.39520118e-02,  2.13391231e-05, -3.46194729e-02, -2.71392957e-04,
        -1.55740476e-03, -1.61484107e-02,  5.34229018e-02, -3.36914361e-02,
        -7.20024481e-02, -2.69948002e-02,  3.88855897e-02, -3.83572700e-03,
        -4.18707989e-02, -3.97722945e-02, -3.50391008e-02, -7.93507919e-02,
         2.63079777e-02,  3.78062092e-02, -3.99650186e-02, -4.33609001e-02,
        -1.55501021e-02, -5.84293902e-03, -7.53786564e-02,  9.21819732e-02,
        -4.20758426e-02, -6.05439097e-02,  1.11310221e-02, -5.01601622e-02,
        -7.00914785e-02, -1.18700445e-01, -4.29972447e-02,  7.87899196e-02,
        -2.83439104e-02,  2.93379053e-02,  4.40595336e-02, -7.98096657e-02,
        -1.01290904e-02,  1.73291322e-02, -2.87935864e-02,  7.82347992e-02,
         1.59652233e-02, -4.74683680e-02, -3.37990560e-02, -9.81367100e-03,
         1.11967884e-03, -1.40191853e-01, -1.21523619e-01,  4.45947535e-02,
         4.09820816e-03, -6.21589459e-02,  9.11579877e-02, -2.62122769e-02,
        -9.16588753e-02, -1.32042738e-02, -5.51475137e-02,  7.03601763e-02,
         1.01966269e-01, -7.84462504e-03,  1.94286834e-02, -5.28861955e-02],
                "k": 8
            }
        }
    }
}